In [2]:
!pip install keras-tuner


     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=0db4c630e0ac69c22b030df1cd0e2109f3935bc93df554e034d3fa27a918bd73
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=d2f86814726dcbd5f1cf48efccd166d7b94ac03e3544e4d0dd70f98778d4b5ff
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [14]:
fashion_mnist=tf.keras.datasets.fashion_mnist

In [15]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [17]:
test_images=test_images/255.0
train_images=train_images/255.0

In [18]:
train_images[0].shape

(28, 28)

In [19]:
train_images.shape

(60000, 28, 28)

In [20]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [27]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [28]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [29]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [30]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3042 - accuracy: 0.0987 - val_loss: 2.3032 - val_accuracy: 0.0942
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3040 - accuracy: 0.0987 - val_loss: 2.3038 - val_accuracy: 0.1008
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3041 - accuracy: 0.0998 - val_loss: 2.3037 - val_accuracy: 0.0942


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3044 - accuracy: 0.0991 - val_loss: 2.3044 - val_accuracy: 0.1008
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3039 - accuracy: 0.0995 - val_loss: 2.3033 - val_accuracy: 0.1003
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3040 - accuracy: 0.0997 - val_loss: 2.3044 - val_accuracy: 0.1008


Epoch 1/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.7870 - accuracy: 0.7049 - val_loss: 0.5464 - val_accuracy: 0.7868
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.5414 - accuracy: 0.7987 - val_loss: 0.5074 - val_accuracy: 0.7977
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4705 - accuracy: 0.8281 - val_loss: 0.4360 - val_accuracy: 0.8353


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.7436 - accuracy: 0.7216 - val_loss: 0.5606 - val_accuracy: 0.7943
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5010 - accuracy: 0.8160 - val_loss: 0.4592 - val_accuracy: 0.8313
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4477 - accuracy: 0.8389 - val_loss: 0.4239 - val_accuracy: 0.8455


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3029 - accuracy: 0.0966 - val_loss: 2.3029 - val_accuracy: 0.0925
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3027 - accuracy: 0.0989 - val_loss: 2.3030 - val_accuracy: 0.0942
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3028 - accuracy: 0.0996 - val_loss: 2.3028 - val_accuracy: 0.0973


INFO:tensorflow:Oracle triggered exit


In [31]:
model=tuner_search.get_best_models(num_models=1)[0]

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        36896     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2359424   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,398,890
Trainable params: 2,398,890
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4177 - accuracy: 0.8489 - val_loss: 0.4177 - val_accuracy: 0.8498
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3949 - accuracy: 0.8568 - val_loss: 0.3929 - val_accuracy: 0.8597
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3791 - accuracy: 0.8633 - val_loss: 0.3715 - val_accuracy: 0.8625
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3645 - accuracy: 0.8682 - val_loss: 0.3743 - val_accuracy: 0.8668
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3525 - accuracy: 0.8724 - val_loss: 0.3759 - val_accuracy: 0.8620
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3417 - accuracy: 0.8759 - val_loss: 0.3865 - val_accuracy: 0.8602
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3312 - accuracy: 0.8795 - val_loss: 0.3502 - val_accuracy